In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import random
import torch
from typing import List, Dict, Optional
import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna
from optuna.pruners import MedianPruner
from optuna.exceptions import TrialPruned
import warnings

In [2]:
sys.path.append('..') 

from src.models.autoencoder import AutoEncoder
from src.models.autoencoder_trainer import *
from src.data.data_utils import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_dir = '../NETFLIX_DATA/partitions/train'
val_dir = '../NETFLIX_DATA/partitions/validation'
checkpoint_dir = '../model_checkpoints'

In [4]:
# retreive training data info
train_partition_files = get_data(data_dir)
print(f"Number of training partitions: {len(train_partition_files)}")
val_partition_files = get_data(val_dir)
print(f"Number of validation partitions: {len(val_partition_files)}")

seed = 42
random.seed(seed)

# testing
sample_train_partitions = random.sample(train_partition_files, 1)

sample_val_partitions = []
for partition in sample_train_partitions:
  val_partition = partition.copy()
  val_partition['path'] = partition['path'].replace('train', 'validation')
  sample_val_partitions.append(val_partition)


print(f"Train EX: {sample_train_partitions[0]}")
print(f"Val EX: {sample_val_partitions[0]}")

Number of training partitions: 34
Number of validation partitions: 34
Train EX: {'path': '../NETFLIX_DATA/partitions/train/part_1_7.parquet', 'part': 1, 'group': 7}
Val EX: {'path': '../NETFLIX_DATA/partitions/validation/part_1_7.parquet', 'part': 1, 'group': 7}


In [5]:
# build user_map and movie_map
user_map, movie_map = map_id(sample_train_partitions)

Mapping IDs: 100%|██████████| 1/1 [00:00<00:00,  4.16it/s]

Map successful for 317577 users, 354 movies


In [6]:
# preload user rating profiles

train_user_data = AutoEncoder.load_user_data(partitions=sample_train_partitions, 
                                             user_map=user_map)

validation_user_data = AutoEncoder.load_user_data(partitions=sample_val_partitions, 
                                                  user_map=user_map)

Building user rating profiles: 100%|██████████| 71258/71258 [00:13<00:00, 5425.97it/s]


In [7]:
# optuna objective
def objective(trial):
    params = {
        "num_epochs": 5,
        "batch_size": 512,
        "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.001, log=True),
        "hidden_dims": trial.suggest_categorical("hidden_dims", 
                                                 [[512,256,128], [256,128], [512,128]]),
        "dropout": trial.suggest_float("dropout", 0.3, 0.7),
        "l2_reg": trial.suggest_float("l2_reg", 0.00001, 0.01, log=True),
        "checkpoint_interval": 5,
        "eval_interval": 1,
    }

                #[[512,256,128], [256,128], [512,128]]

    try:
        model, rmse = train_autoencoder(
            train_partitions=sample_train_partitions,
            user_map=user_map,
            movie_map=movie_map,
            validation_partitions=sample_val_partitions,
            checkpoint_dir=checkpoint_dir,
            trial=trial,
            user_data=train_user_data,
            validation_data=validation_user_data,
            **params
        )

        return float(rmse)
    except optuna.TrialPruned:
        raise # reraise prune error for tuning

In [8]:
# tuning
warnings.filterwarnings("ignore", module="optuna.*")

study = optuna.create_study(
    study_name="autoencoder_tuning", 
    direction='minimize',
    
    # prune after 2 trials, after 1 if really bad
    pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
    
    sampler=optuna.samplers.TPESampler(),
    storage="sqlite:///optuna_study.db",
    load_if_exists=True
)

study.optimize(objective, n_trials=10, timeout=2*3600)

[I 2025-06-02 09:17:46,554] A new study created in RDB with name: autoencoder_tuning


Device: cuda


Calculating global mean: 100%|██████████| 1/1 [00:00<00:00,  8.21it/s]

Global mean rating: 3.583



Epoch 1/3: 100%|██████████| 311/311 [00:21<00:00, 14.34it/s, loss=1.3305]


...

Epoch 1 - Average Loss: 6.5927
DEBUG: Final - num_batches: 92, total_predictions: 100561
DEBUG: loss 2.596964957921401, rmse 1.59963049616769, mae 1.3640244320695716
Validation | Loss: 2.5970, RMSE: 1.5996


Epoch 2/3: 100%|██████████| 311/311 [00:19<00:00, 15.75it/s, loss=1.8408]


Epoch 2 - Average Loss: 2.0834


[I 2025-06-02 09:27:30,258] Trial 9 pruned. 


DEBUG: Final - num_batches: 92, total_predictions: 100561
DEBUG: loss 1.318318934544273, rmse 1.1317986374580662, mae 0.9118149209433396


In [9]:
print("Best trial")
print(f"RMSE: {study.best_value:.4f}")
print(f"Params: {study.best_params}")

Best trial
RMSE: 0.9446
Params: {'learning_rate': 0.0009560093645864017, 'hidden_dims': [128, 64], 'dropout': 0.4909998063083557, 'l2_reg': 0.0002766435018640519}
